In [1]:
from textFeatures import *

In [2]:
import json
def raw_feature(oufname):
    inf = open(oufname, 'r')
    features = []
    lbs = []
    for line in inf:
        js = json.loads(line)
        lb = 1 if (js["votes"]["funny"]+js["votes"]["cool"]+js["votes"]["useful"] > 0) else 0
        features.append(js['text']) #Replace this line with the feature you want
        lbs.append(lb)
    return features, lbs
texts_train, Y_train = raw_feature("../data/train.json")
texts_test, Y_test = raw_feature("../data/test.json")
print "Finished"

Finished


In [3]:
print "Building tfidf model ..."
X_train_tfidf, vectorizer = tfidf_extraction(texts=texts_train, ngram_range=(1,1), max_features=1500)
print "Fitting testset to tfidf model ..."
X_test_tfidf, vectorizer = tfidf_extraction(texts_test, vectorizer, ngram_range=(1,1), max_features=1500)
print "Finished"

Building tfidf model ...
Fitting testset to tfidf model ...
Finished


In [4]:
print "Select feature for training set via Chi2 ..."
X_train_tfidf_ch2, ch2 = feature_select_chi2(X_train_tfidf, Y_train, k=20)
print "Select feature for testset via Chi2 ..."
X_test_tfidf_ch2, ch2 = feature_select_chi2(X_test_tfidf, Y_test, ch2)
print "Finished"

Select feature for training set via Chi2 ...
Select feature for testset via Chi2 ...
Finished


In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
print "Building AdaBoost Model ..."
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),algorithm="SAMME",n_estimators=200)
bdt.fit(X_train_tfidf_ch2.toarray(), Y_train)

Y_pre = bdt.predict(X_test_tfidf_ch2.toarray())
report = classification_report(Y_test, Y_pre)
print report

Building AdaBoost Model ...
             precision    recall  f1-score   support

          0       0.53      0.90      0.67     15019
          1       0.68      0.21      0.32     14981

avg / total       0.61      0.56      0.50     30000



In [13]:
from sklearn.linear_model import SGDClassifier
print "Buiding ElasticNet Model ..."
en = SGDClassifier(loss="squared_hinge", penalty='elasticnet', l1_ratio=0.20, n_iter=20, alpha=0.001)
en.fit(X_train_tfidf_ch2, Y_train)
print "Predicting ..."
Y_pre = en.predict(X_test_tfidf_ch2)
print Y_pre
report = classification_report(Y_test, Y_pre)
print report

Buiding ElasticNet Model ...
Predicting ...
[0 0 1 ..., 1 0 0]
             precision    recall  f1-score   support

          0       0.57      0.63      0.60     15019
          1       0.59      0.52      0.55     14981

avg / total       0.58      0.58      0.58     30000



In [6]:
y_pre_postag, res_svm = my_SVM(X_train_tfidf_ch2, X_test_tfidf_ch2, Y_train, Y_test)
print res_svm

	-- SVM training ...
	-- SVM predicting ...
             precision    recall  f1-score   support

          0       0.54      0.79      0.64     15019
          1       0.61      0.33      0.43     14981

avg / total       0.58      0.56      0.54     30000



In [ ]:
a = vectorizer.get_feature_names()
print a